In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
import time
from skimage.feature import hog
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [35]:
def get_hog_features(img, orient, pix_per_cells, cell_per_block, vis=False, feature_vec=True):
    if vis:
        feature, hog_image = hog(img, orientations=orient, 
                                 pixels_per_cell=(pix_per_cells, pix_per_cells),
                                 cells_per_block=(cell_per_block, cell_per_block), 
                                 visualize=vis, 
                                 feature_vector=feature_vec, 
                                 transform_sqrt=True,
                                 block_norm="L2-Hys")
        
        return feature, hog_image
    else:
        feature = hog(img, orientations=orient, 
                      pixels_per_cell=(pix_per_cells, pix_per_cells), 
                      cells_per_block=(cell_per_block, cell_per_block), 
                      visualize=vis, 
                      feature_vector=feature_vec, 
                      transform_sqrt=True, 
                      block_norm="L2-Hys")
        return feature
                                 

In [139]:
def extract_features(imgs, cspace="RGB", orient=9, pix_per_cells=8, cell_per_block=2, hog_channel=0):
    features = []
    for file_path in imgs:
        image = mpimg.imread(file_path)
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        
        if hog_channel == "ALL":
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], orient, 
                                                     pix_per_cells, 
                                                     cell_per_block, 
                                                     vis=False, 
                                                     feature_vec=True))
            hog_features = np.ravel(hog_features) 
        else:
            hog_features = get_hog_features(feature_image[:, :, hog_channel], orient, 
                                            pix_per_cells, 
                                            cell_per_block,
                                            vis=False, 
                                            feature_vec=True)
        
        
        features.append(hog_features)
    return features
            
                                

In [140]:
images = glob.glob("cars/*/*/*.jpeg")
cars=[]
not_cars = []

for image in images:
    if "image" in image or "extra" in image:
        not_cars.append(image)
    else:
        cars.append(image)
n_sample = 500
cars = cars[:n_sample]
not_cars = not_cars[:n_sample]

In [200]:
cspace = 'HSV'
pix_per_cell = 8
cell_per_block = 1
orient = 9
hog_channel = "ALL"

In [201]:
t1 = time.time()
car_features = extract_features(cars, cspace=cspace, orient=orient,
                                pix_per_cells=pix_per_cell, 
                                cell_per_block=cell_per_block, hog_channel=hog_channel)

notcar_features = extract_features(not_cars, cspace=cspace, orient=orient, 
                                   pix_per_cells=pix_per_cells, 
                                   cell_per_block=cell_per_block, hog_channel=hog_channel)
t2 = time.time()

print(round(t2-t1, 2), " seconds to extract hog features...")

8.48  seconds to extract hog features...


In [202]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)
print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 1 cells per block
Feature vector length: 1728


In [203]:
svc = LinearSVC()
t1 = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t1, 2), 'Seconds to train SVC...')

0.09 Seconds to train SVC...


In [204]:
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t1=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t1, 5), 'Seconds to predict', n_predict,'labels with SVC')

Test Accuracy of SVC =  0.995
My SVC predicts:  [1. 1. 0. 1. 1. 1. 0. 1. 0. 0.]
For these 10 labels:  [1. 1. 0. 1. 1. 1. 0. 1. 0. 0.]
0.01563 Seconds to predict 10 labels with SVC
